In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importing Libraries
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator

2023-02-09 11:39:45.194705: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-09 11:39:45.360316: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-09 11:39:45.360337: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-09 11:39:46.135180: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [3]:
# Data generation for training data
train_data = ImageDataGenerator(horizontal_flip=True,
                                vertical_flip=False,
                                rescale=1./255,
                                shear_range=0.2,
                                zoom_range=0.2)

In [4]:
# Training Dataset
train = train_data.flow_from_directory(directory ='/home/ubuntu/Desktop/Face_Recognition_DS_team/Train/',
                                       target_size=(224, 224),
                                       batch_size = 32,
                                       class_mode ="categorical")   

Found 10564 images belonging to 9 classes.


In [5]:
#validation dataset
validation = train_data.flow_from_directory(directory ='/home/ubuntu/Desktop/Face_Recognition_DS_team/Train/',
                                       target_size=(224, 224),
                                       batch_size = 32,
                                       class_mode ="categorical")

Found 10564 images belonging to 9 classes.


In [6]:
# Checking classes
train.class_indices

{'arun': 0,
 'chaitanya': 1,
 'laxmi': 2,
 'navin': 3,
 'papu': 4,
 'priti': 5,
 'rahul': 6,
 'sangram': 7,
 'sanket': 8}

# MobileNetV2

In [7]:
base_model = keras.applications.MobileNetV2(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

2023-02-09 11:39:47.723824: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-09 11:39:47.723863: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-09 11:39:47.723896: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ubuntu-Zako): /proc/driver/nvidia/version does not exist
2023-02-09 11:39:47.724902: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
base_model.trainable = False

In [9]:
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)

#pooling layer 
x = keras.layers.GlobalAveragePooling2D()(x)

#final dense layer
outputs = keras.layers.Dense(9, activation = 'softmax')(x)

model = keras.Model(inputs,outputs)

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 9)                 11529     
                                                                 
Total params: 2,269,513
Trainable params: 11,529
Non-trainable params: 2,257,984
_________________________________________________________________


In [11]:
model.compile(optimizer='adam',loss=['categorical_crossentropy'],metrics=['accuracy'])

In [12]:
from tensorflow.keras.callbacks import EarlyStopping
# Early stopping to avoid overfitting
es = EarlyStopping(monitor = "accuracy",patience = 3)

In [ ]:
%%time
model.fit(train, validation_data = validation, epochs=20, callbacks = [es], steps_per_epoch = 10)

Epoch 1/20


2023-02-09 11:39:50.166927: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 19267584 exceeds 10% of free system memory.
2023-02-09 11:39:53.051475: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 19267584 exceeds 10% of free system memory.
2023-02-09 11:39:53.065196: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 51380224 exceeds 10% of free system memory.
2023-02-09 11:39:53.183772: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 51380224 exceeds 10% of free system memory.
2023-02-09 11:39:53.241580: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 51380224 exceeds 10% of free system memory.


10/10 [==============================] - 323s 35s/step - loss: 2.2099 - accuracy: 0.2219 - val_loss: 1.8952 - val_accuracy: 0.3321
Epoch 2/20
10/10 [==============================] - 306s 34s/step - loss: 1.6310 - accuracy: 0.4625 - val_loss: 1.4301 - val_accuracy: 0.5523
Epoch 3/20
10/10 [==============================] - 342s 38s/step - loss: 1.2870 - accuracy: 0.6313 - val_loss: 1.1040 - val_accuracy: 0.7181
Epoch 4/20
10/10 [==============================] - 557s 62s/step - loss: 1.1282 - accuracy: 0.7125 - val_loss: 0.9285 - val_accuracy: 0.7841
Epoch 5/20
10/10 [==============================] - 469s 52s/step - loss: 0.8811 - accuracy: 0.7875 - val_loss: 0.7910 - val_accuracy: 0.8233
Epoch 6/20
10/10 [==============================] - ETA: 0s - loss: 0.7391 - accuracy: 0.8469

In [ ]:
model.save("model.h5")